In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random
import mysql.connector
from datetime import datetime

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (iPad; CPU OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.96 Mobile Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/83.0.478.37",
    "Mozilla/5.0 (Linux; Android 10; SM-G973F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.96 Mobile Safari/537.36"
]
def create_driver_with_user_agent(user_agent):
    options = Options()
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument(f'user-agent={user_agent}')
    return webdriver.Chrome(options=options)

def go_to_user_page(user_url):
    driver = create_driver_with_user_agent(random.choice(user_agents))
    driver.get(user_url)
    time.sleep(3)
    try:
        cookies = "/html/body/div[4]/div/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[2]"
        cookies2 = "/html/body/div[3]/div/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[2]"
        try:
            cookies_button = driver.find_element(By.XPATH, cookies)
        except:
            cookies_button = driver.find_element(By.XPATH, cookies2)
        cookies_button.click()
        time.sleep(2)
    except:
        driver.close()
        return None
    return driver

def connect_to_db():
    db_params = {
        'host': '192.168.100.200',
        'user': 'ecorradini',
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    return db_connection,db_cursor

def fetch_users():
    db_connection,db_cursor = connect_to_db()
    query = """
        SELECT DISTINCT user
        FROM data d
        WHERE d.user != '' AND NOT EXISTS (
            SELECT 1
            FROM user u
            WHERE d.user = u.url
        );
    """
    db_cursor.execute(query)
    return db_cursor.fetchall()

def insert_user(url, username, displayname, bio, followers, bio_url):
    db_params = {
        'host': '192.168.100.200',  # Or your database host
        'user': 'ecorradini',   # Your database username
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    
    if bio is not None:
        b = bio.replace('"','\"').replace("'","\'")
    else:
        b = bio
    insert_query = "INSERT IGNORE INTO user (url, username, displayname, bio, followers, bio_url) VALUES (%s, %s, %s, %s, %s, %s)"
    data = (url,username,displayname,bio,followers,bio_url)
    db_cursor.execute(insert_query, data)
    db_connection.commit()
    
    db_cursor.close()
    db_connection.close()

In [2]:
base_path = "/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[1]"

def get_username(driver):
    username_path = base_path + "/div[1]/div[1]/div[2]/div/span/span"
    try:
        username = driver.find_element(By.XPATH,username_path).text
        return username
    except:
        return None
    
def get_displayname(driver):
    displayname_path = base_path + "/div[1]/div[1]/h2"
    try:
        displayname = driver.find_element(By.XPATH,displayname_path).text
        return displayname
    except:
        return None
    
def get_bio(driver):
    bio_path = base_path + "/div[2]/span"
    try:
        bio = driver.find_element(By.XPATH,bio_path).text
        return bio
    except:
        return None
    
def get_followers(driver):
    followers_path = base_path + "/div[3]/div[1]/a/div/span/span"
    try:
        followers = int(driver.find_element(By.XPATH,followers_path).text.replace(",",""))
        return followers
    except:
        followers_path2 = base_path + "/div[2]/div[1]/a/div/span/span"
        try:
            followers2 = int(driver.find_element(By.XPATH,followers_path2).text.replace(",",""))
            return followers2
        except:
            return None

def get_bio_url(driver):
    bio_url_path = base_path + "/div[3]/div[2]/div/span[2]/a/span/span"
    try:
        bio_url = driver.find_element(By.XPATH,bio_url_path).text
        return bio_url
    except:
        return None

In [ ]:
while True:
    users = [u[0] for u in fetch_users()]
    if len(users) > 0:
        url = users[0]
        try:
            driver = go_to_user_page(url)
            username = get_username(driver)
            displayname = get_displayname(driver)
            bio = get_bio(driver)
            followers = get_followers(driver)
            bio_url = get_bio_url(driver)
            insert_user(url, username, displayname, bio, followers, bio_url)
            driver.close()
        except Exception as e:
            continue

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json): error trying to connect: dns error: failed to lookup address information: Try again); using driver found in the cache


In [33]:
users = [u[0] for u in fetch_users()]
driver = go_to_user_page(users[5])

In [34]:
print(get_username(driver))
print(get_displayname(driver))
print(get_bio(driver))
print(get_followers(driver))
print(get_bio_url(driver))

096efra_cp
Cañas M. Efrain
Encouragement
4
None
